# Imports

In [2]:
# importing required libraries
import pandas as pd
import numpy as np
import category_encoders as ce
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
%matplotlib qt

import re
from datetime import datetime as dt
import calendar as cl
from scipy.stats import pearsonr, median_absolute_deviation as mad, chi2
from math import exp
import random
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from statsmodels.stats.stattools import medcouple as calc_skew
from sklearn.decomposition import PCA
import seaborn as sns

# ML
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors as NN, LocalOutlierFactor as LOF
from sklearn.metrics import r2_score
from sklearn.covariance import MinCovDet
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split as tvt
from patsy import dmatrices
import statsmodels.api as sm

# Notebook settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
 
# Silence chained warnings
pd.options.mode.chained_assignment = None
import logging
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    logger.setLevel(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

# Read Data

In [3]:
# Read insurance data
ins = pd.read_csv('/Users/mfairb/Documents/MSA/AA503/Spring2/Fraud Analytics/Code/aggregated_policy_nolabel.csv')
ins_l = pd.read_csv('/Users/mfairb/Documents/MSA/AA503/Spring2/Fraud Analytics/Code/aggregated_policy_label.csv')

# Unsupervised Learning

## Univariate Techniques

### Benford's Law

In [4]:
from benfordslaw import benfordslaw

# Initialize
bl = benfordslaw(alpha=0.05)

# Extract election information.
X = ins['Reward_Amount'].values
results = bl.fit(X)

# Plot
bl.plot(title='Benfords Law');

# The chi-square test is a test of the null hypothesis that the data follows Benford's Law

[benfordslaw] >Analyzing digit position: [1]
[benfordslaw] >[chi2] Anomaly detected! P=0, Tstat=13510.2


### Z-Scores & Robust Z-Scores
Using Z-scores works well with symmetric distributions (particularly normal) (because means and std. devs work well with symmetric distributions). HOWEVER - since we're trying to detect outliers, it doesn't necessarily make sense to use Z-score since normal distributions are heavily influence by outliers. Instead, use Robust Z-Scores.

In [5]:
# Start with distribution of variable
ins = ins.rename(columns={'Coverage_Income_Ratio_Claim':'CIRC'})

plt.hist(ins.CIRC)
plt.title('CIRC')
plt.show();

# Plot Z-scores
ins['Z_CIRC'] = abs((ins.CIRC - np.mean(ins.CIRC)) / np.std(ins.CIRC))
plt.hist(ins.Z_CIRC)
plt.title('CIRC Z-scores')
plt.show();

# Plot robust Z-scores
ins['RZ_CIRC'] = abs((ins.CIRC - np.median(ins.CIRC)) / mad(ins.CIRC))
plt.hist(ins.RZ_CIRC)
plt.title('CIRC Robust Z-scores')
plt.show();

### Box-Plots
1.5*IQR rule work best for symmetric distributions and often reports large number of outliers when skewed. 

In [ ]:
# Adjusted boxplot - No python capability for this
# Standard box-plot
plt.boxplot(ins.CIRC)
plt.title('CIRC Boxplot')
plt.show();

## Multivariate Techniques

### Mahalanobis Distances
Generalization of z-scores to multi-dimensional space. Essentially done by replacing the univariate mean with the multivariate mean and the standard deviation with the covariance matrix

***NOTE***: I chose to use the MinCovDet function from sklearn.covariance to compute the minimum covariance determinant. This function is known to contain bugs and has significant shortcomings compared to the covMcd function from R's robustbase library. As far as I've researched, there is no better way to compute the MCD in python

In [23]:
# Function to get eigen values from covariance matrix
def get_eigens(cov):
    eig_values, eig_vectors = np.linalg.eig(cov) 
    l1 = eig_values[0]
    l2 = eig_values[1]
    return l1, l2

# Plot
x = ins.Time_Between_CL_R
y = ins.Cov_Limit_Claim/1000

# Plot x & y
plt.scatter(x, y)
plt.xlabel('Time Between Claim and Payment')
plt.ylabel('Coverage Limit at Claim (Thousands $)')
plt.title('Coverage Limit vs. Speed of Payment')
ax = plt.gca()

# Get covariance matrix and resulting eigen values
cov = np.cov(x, y)
l1, l2 = get_eigens(cov)

# Calculate 
ppf = chi2.ppf(0.9999975, 2)
w = 2*np.sqrt(l1*ppf)
h = 2*np.sqrt(l2*ppf)
ell = Ellipse(xy=(x.mean(), y.mean()),
              width=w,
              height=h,
              edgecolor='b', fc='None', lw=2)

ax.add_patch(ell)
ax.set_xlim(min(x)-3, max(x)+1)
ax.set_ylim(min(y)-100, max(y)+200);

# Calculate robust version of Mahalanobis Distance
rmd = pd.DataFrame()
rmd['x'] = x
rmd['y'] = y

# Get covariance matrix and resulting eigen values
cov = MinCovDet(random_state=0).fit(rmd)
robust_cov = cov.covariance_
rl1, rl2 = get_eigens(robust_cov)

# Calculate
robust_center = cov.location_
ppf = chi2.ppf(0.9999975, 2)
rw = 2*np.sqrt(rl1*ppf)
rh = 2*np.sqrt(rl2*ppf)
robust_ell = Ellipse(xy=(robust_center[0], robust_center[1]),
             width=rw,
             height=rh,
             edgecolor='r', fc='None', lw=2)

ax.add_patch(robust_ell);

### K-Nearest Neighbors
Still can be bothered by outliers since std. mean and covariance matrix used<br>
kNN is good at detecting global outliers but not great at detecting local outliers

In [ ]:
# KNN on Time Between Claim ratio and Coverage limit claim
kdf = pd.DataFrame()
kdf['TBCR'] = ins.Time_Between_CL_R
kdf['CLC'] = ins.Cov_Limit_Claim/1000

# Fit KNN
knn = NN(n_neighbors=5)
knn.fit(kdf)

# Get KNN score
distances, indices = knn.kneighbors()

# Get average distance to k nearest neighbros
kdf['distances'] = [listy.mean() for listy in distances]

# Plot KNN score
plt.scatter(kdf.TBCR, kdf.CLC, s=kdf.distances+0.01)
plt.title('Coverage Limit vs. Speed of Payment')
plt.show();

### Local Outlier Factor (LOF)
Ratio of average density of the kNN of an obs. to the density of the observation. Density here is essentially how far you have to travel to get to the nearest point. LOF calculates how far away a given point is from the nearest point and how far the nearest 5 observations are from that point. An LOF > 1 implies more anamolous.

In [12]:
# Create LOF df
ldf = pd.DataFrame()
ldf['TBCR'] = ins.Time_Between_CL_R
ldf['CLC'] = ins.Cov_Limit_Claim/1000

# Fit LOF
lof = LOF(n_neighbors=5)

# Create labels: 1 for inlier, -1 for outlier
labels = lof.fit_predict(ldf)

# Get Negative outlier scores: "The higher, the more normal. 
# Inliers tend to have a LOF score close to 1 (negative_outlier_factor_ close to -1), 
# while outliers tend to have a larger LOF score" -sklearn)
nos = -lof.negative_outlier_factor_

outlier_index = np.where(labels==-1)[0]
outliers = ldf[ldf.index.isin(outlier_index)]
inliers = ldf[~ldf.index.isin(outlier_index)]

# Plot points colored by label
plt.scatter(inliers.TBCR, inliers.CLC)
plt.scatter(outliers.TBCR,outliers.CLC, color='r')
plt.title('Coverage Limit vs. Speed of Payment: Colored')
plt.show();

# Would like to implement but some values are too large. There may very well be a 
# way around this but would need to research
# Plot points with size-adjusted by LOF 
# plt.scatter(ldf.TBCR, ldf.CLC, s=nos)
# plt.title('Coverage Limit vs. Speed of Payment: Size-Adjusted')
# plt.show();

### Isolation Forest
Randomly split along an axis (pick a random axis and a random point) and split there. Then, split again within splits that were already made. The easier it is to split a point, the more likely it is to be anamolous. Then standardize count of splits since algo would be different every time run. This is a single tree. To make a forest, perform many times and take average score of many trees.

In [ ]:
# Create df for Isolation Forest
idf = pd.DataFrame()
idf['TBCR'] = ins.Time_Between_CL_R
idf['CLC'] = ins.Cov_Limit_Claim/1000

# Fit Isolation Forest
iso_100 = IsolationForest(random_state=273,
                          n_estimators=100,
                          max_samples=1000).fit(idf)
iso_500 = IsolationForest(random_state=273,
                          n_estimators=500,
                          max_samples=5000).fit(idf)

# Get anomaly scores on train
iso_100_scores = -iso_100.score_samples(idf)
iso_500_scores = -iso_100.score_samples(idf)

# Predict labels on train
iso_100_labels = iso_100.predict(idf)
iso_500_labels = iso_500.predict(idf)

# Put in df
idf['iso_100_score'] = iso_100_scores
idf['iso_500_score'] = iso_500_scores
idf['iso_100_labels'] = iso_100_labels
idf['iso_500_labels'] = iso_500_labels

In [ ]:
# Plot anomaly scores
plt.hist(iso_100_scores);
plt.title('Coverage Limit vs. Speed of Payment')
plt.show();
plt.hist(iso_500_scores)
plt.title('Coverage Limit vs. Speed of Payment')
plt.show();

# Plot labels
plt.scatter(idf.TBCR, idf.CLC, c=iso_100_labels)
plt.show();

# Plot scores
plt.scatter(idf.TBCR, idf.CLC, s=iso_100_scores)
plt.show();

In [ ]:
plt.hist(iso_100_scores);
plt.hist([exp(s) for s in iso_100_scores]);
plt.hist(iso_100_scores**2);

### PCA

In [3]:
# Keep a few columns
df = ins[['Reward_Amount', 'Time_Between_CL_R', 'Time_Between_IN_CL', 'Med_Conditions']]
df['Med_Conditions'] = [4 if m == '3+' else m for m in df.Med_Conditions] # Cleaning step for ease of PCA

# Scale data
df_scaled = StandardScaler().fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)

# Perform PCA
pca = PCA()
pcs = pd.DataFrame(pca.fit_transform(df), columns=[f'PC{i}' for i in range(1, 5)])

# View amount of variance explained
i = 1
for v in pca.explained_variance_ratio_:
    print(f'Variance explained by PC {i} = {round(v, 4)}')
    i+=1

Variance explained by PC 1 = 0.9999
Variance explained by PC 2 = 0.0001
Variance explained by PC 3 = 0.0
Variance explained by PC 4 = 0.0


In [21]:
# # Plot initialisation
def plot_pcs(pcs, labels):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(pcs['PC1'], pcs['PC2'], pcs['PC3'], c=labels, cmap="Set2_r", s=60)
    plt.show();
     
    # label the axes
    ax.set_xlabel("PC1")
    ax.set_ylabel("PC2")
    ax.set_zlabel("PC3")
    ax.set_title("PCA on the iris data set")
    plt.show();


In [4]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

fig = plt.figure()
ax = fig.gca(projection='3d')

scale = 8
# Make data.
X = np.arange(-scale, scale, 0.25)
Y = np.arange(-scale, scale, 0.25)
X, Y = np.meshgrid(X, Y)
Z = X**2 + Y**2

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                   linewidth=0, antialiased=False)

# Customize the z axis.
ax.set_zlim(0, 100)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# rotate the axes and update
for angle in range(0, 360):
   ax.view_init(30, 40)

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show();